In [50]:
#Импортирование необходимых библиотек
import re
import os
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from navec import Navec

In [109]:
#Сформируем класс датасет
class TextDataset(data.Dataset):
    def __init__(self, path, embedding_navec, prev_words):
        self.prev_words = prev_words
        self.path = path
        self.embedding_navec = embedding_navec

        with open(self.path, "r", encoding="utf8") as file:
            [file.readline() for _ in range(15)] #Пропустим первые 15 строк, так как они являются мусорными в нашем тексте
            self.text = file.read()
            self.text = self.text.replace('\ufeff', '')  # убираем невидимый символ
            self.text = self.text.replace('\n', ' ')
            self.text = re.sub(r'[^А-яA-z- ]', '', self.text)  # Отчищаем текст, от мусорных символов
            self.text = self.text.lower().split() #Формируем список слов

        self.all_words = [word for word in self.text if word in self.embedding_navec] #Все слова, которые есть в navec
        self.unique_words = sorted(set(self.all_words)) #Уникальные слова, отсортированные
        self.int_in_word = dict(enumerate(self.unique_words)) #Словарь число:слово
        self.word_in_int = {b: a for a, b in self.int_in_word.items()} #Словаро слово:число
        self.len_unique_words = len(self.unique_words) #Количество уникальных слов
        self.len_all_words = len(self.all_words) #Количество слов
        self.ohe = torch.eye(self.len_unique_words) #Векторы кодирования целевого значения

    def __getitem__(self, item):
        data = torch.vstack([torch.tensor(self.embedding_navec[self.all_words[index]]) for index in range(item, item+self.prev_words)])
        target = self.word_in_int[self.all_words[item+self.prev_words]]
        return data, target

    def __len__(self):
        return self.len_all_words - 1 - self.prev_words

In [131]:
#Сформируем класс нейронной сети
class WordsNN(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.hidden_size = 256
        self.in_features = in_features
        self.out_features = out_features

        self.rnn = nn.RNN(in_features, self.hidden_size, batch_first=True)
        self.out = nn.Linear(self.hidden_size, out_features)

    def forward(self, x):
        return self.out(self.rnn(x)[1])

In [181]:
navec = Navec.load("navec_hudlit_v1_12B_500K_300d_100q.tar")
d_train = TextDataset(path="text_2.txt", embedding_navec=navec, prev_words=10)
train_data = data.DataLoader(d_train, batch_size=64, shuffle=True)
len(d_train)

3315

In [133]:
#Сформируем процесс обучения нейронной сети
eph = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#Формируем выборку
navec = Navec.load("navec_hudlit_v1_12B_500K_300d_100q.tar")
d_train = TextDataset(path="Plen_Aleksandra_Podarok_(SI)_Litmir.net_228863_c29a0.txt", embedding_navec=navec, prev_words=10)
train_data = data.DataLoader(d_train, batch_size=64, shuffle=True)
#Сформируем объект модели
model = WordsNN(in_features=300, out_features=d_train.len_unique_words)
model.train()
model.to(device)
#Зададим оптимизаторы и функцию потерь
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=0.0005, weight_decay=0.0001)
#Зададим процесс обучения
for _e in range(eph):
    lm_count, loss_mean = 0, 0
    tqdm_train_data = tqdm(train_data, leave=True)
    for x_train, y_train in tqdm_train_data:
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        predict = model(x_train).squeeze(0)
        loss = loss_func(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1 / lm_count * loss.item() + (1 - 1 / lm_count) * loss_mean

        tqdm_train_data.set_description(f"Текущая эпоха: [{_e+1}/{eph}]  Текущая ошибка: {round(loss_mean, 4)}")

Текущая эпоха: [55/100]  Текущая ошибка: 0.1358:  85%|██████████████████████████▍    | 285/334 [00:05<00:00, 56.61it/s]


KeyboardInterrupt: 

In [135]:
#Остановим процесс обучения, так как был достигнут локальный минимум, сохраним модель
state_dict = model.state_dict()
torch.save(state_dict, "model_rnn_words.tar")
print("Модель была успешно сохранена!")

Модель была успешно сохранена!


In [ ]:
#Сделаем предсказание модели
model.eval()
model.to("cpu")
predict = "что на высоте десяти километров сотовую связь телефону не обнаружить никак и все это бесполезно"
total = 20
for _ in range(total):
    data = 